SET PARAMETERS

In [ ]:
# initialize params
DEBUG = True
saveReport = True
toPrint = True
reportName = 'notebook'
txt_label = "Classification of integrated c1 and c2 CNVs samples"


# train test split params
split_train_size = 40
split_random_state = 0

# classification params
classification_args = {
    "n_splits": 10,
    "random_state": 0
}

In [ ]:
# plotting params

with_swarm = False
highRes = False
if highRes:
    img_ext = '.pdf'
else:
    img_ext = '.png'


cnv_plot_kwargs = {
    "vmin": -2,
    "vmax": +2,
    "mincol": "red",
    "midcol": "white",
    "maxcol": "blue",
    "function_dict": None
}

var_plot_kwargs = {
    "vmin": 0,
    "vmax": 4,
    "mincol": "white",
    "midcol": "orange",
    "maxcol": "red",
    "function_dict": {
        "no mutation": 0,
        "missense": 1,
        "nonframeshiftIndel": 2,
        "nonsense": 3,
        "frameshiftIndel": 4
    }
}

mixed_plot_kwargs = {
    "vmin": -4,
    "vmax": +4,
    "mincol": "red",
    "midcol": "white",
    "maxcol": "purple",
    "function_dict": None
}

In [ ]:
# data file
cnv_data_fpath = "output/headneck/integrate_cohorts/c1c2/CNV_mapped_filt/integrated_data.csv"
var_data_fpath = "output/headneck/integrate_cohorts/c1c2/genepanel/integrated_data.csv"

# sample_info file
sample_info_fpath = "output/headneck/integrate_cohorts/c1c2/integrated_sample_info.csv"
sample_class_column = "Relapsed"
class_labels = ["relapsed","NOTrelapsed"]
class_values = [1,0]

# genes_info file
genes_info_fpath = "output/headneck/setup_c1_oncoscan_byNexus/genes_info.csv"
chr_col = 'chr_int'
gene_id_col = 'gene'

# output dir
output_directory = "output/headneck/classification/"+reportName

In [ ]:
# arguments to load the sample_info file
sample_info_read_csv_kwargs = {
    "sep": "\t",
    "header": 0,
    "col_as_index":"patientID"
}

In [ ]:
# select features
genepanel_fpath = "output/headneck/setup_c1_genepanel/process_select_primary/data_processed.csv"
genepanel_key_name = 'genepanel'

feature_dirs = ['c1_prmr_OncFltNxEx', 'c2_ExcvFltNxEx', 'c1_prmr_mapped_c2_CnvNxEx', 'c1_prmr_mapped_c2_Cnv', 'c1_prmr_mapped_c2_CnvMixedNxEx']
feature_key_names = ['c1_OncFltNxEx', 'c2_ExcvFltNxEx', 'c3_CnvNxEx', 'c3_Cnv', 'c3_CnvMixedNxEx']

SET ENVIRONMENT

In [ ]:
# custom imports
from omics_processing.io import (
    set_directory, load_clinical
)
from omics_processing.remove_duplicates import (
    remove_andSave_duplicates
)
from gene_signatures.core import (
    custom_div_cmap,
    get_chr_ticks,
    choose_samples,
    parse_arg_type,
    boxplot,
    set_heatmap_size,
    set_cbar_ticks,
    edit_names_with_duplicates,
    plot_confusion_matrix,
    define_plot_args,
    plot_scatter_scores,
    plot_roc_with_std_for_one_model,
    plot_roc_for_many_models,
    compute_and_plot_confusion_matrices,
    plot_prediction_counts_per_class,
    plot_data_heatmap,
    extract_gene_set,
    save_image,
    check_path_integrity
)

# basic imports
import os, sys
import numpy as np
import pandas as pd
import json
from scipy.spatial.distance import pdist, squareform
from natsort import natsorted, index_natsorted
import math
import logging
from sklearn import linear_model
from sklearn import svm
from distutils.util import strtobool
from scipy.stats import binom_test
from sklearn.externals import joblib
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_curve, auc
from scipy import interp

# plotting imports
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

script_path = os.getcwd()
logger = logging.getLogger(__name__)

Functions

In [ ]:
def _run_classification(
        dat, dat_target, random_state=None, n_splits=10):

    min_class_count = np.unique(dat_target, return_counts=True)[1].min()
    if n_splits is not None:
        if (n_splits > dat.shape[0]) or (n_splits > min_class_count):
            n_splits = min_class_count
    if random_state is not None:
        random_state = parse_arg_type(random_state, int)
    else:
        random_state = 0
    logger.info(
        "model: svm.LinearSVC with l2 penalty, squared_hinge loss " +
        "and random_state: "+str(random_state)
    )
    model = svm.LinearSVC(
        penalty='l2', C=1, random_state=random_state,
        loss='squared_hinge', dual=False
    )

    logger.info("Running classification...")
    dat = dat.copy()
    dat_target = dat_target.copy()

    X = dat
    y = dat_target
    k_fold = StratifiedKFold(n_splits=n_splits)
    cross_val_scores = []
    all_coefs = np.zeros((n_splits, dat.shape[1]))
    y_train_predictions = pd.Series(index=y.index)
    y_train_predictions.name = "train_predictions"
    
    fprs = []
    tprs = []
    interps = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    split_i = 0
    for train_indices, test_indices in k_fold.split(X, y):
        X_train = dat.iloc[train_indices]
        y_train = dat_target.iloc[train_indices]
        
        X_crossval = dat.iloc[test_indices]
        y_crossval = dat_target.iloc[test_indices]

        model.fit(X_train, y_train)
        all_coefs[split_i:split_i+1, :] = model.coef_[0]
        cross_val_scores.append(model.score(X_crossval, y_crossval))
        y_train_predictions.iloc[test_indices] = model.predict(X_crossval)
        
        
#         y_proba = model.decision_function(X_crossval)
        clf = CalibratedClassifierCV(base_estimator=model, cv='prefit')
        clf.fit(X_crossval, y_crossval)
        y_proba = clf.predict_proba(X_crossval)
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(y_crossval, y_proba[:, 1])
#         fpr, tpr, thresholds = roc_curve(y_crossval, y_proba)
        fprs.append(fpr)
        tprs.append(tpr)
        interps.append(interp(mean_fpr, fpr, tpr))
        interps[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)

        split_i += 1

    X = dat
    y = dat_target
    model.fit(X, y)

    all_coefs = pd.DataFrame(all_coefs, columns=dat.columns.values)

    return model, all_coefs, y_train_predictions, cross_val_scores, fprs, tprs, interps, aucs


START ANALYSIS

In [ ]:
# properly set file paths
try:
    os.path.exists(MainDataDir)
except:
    MainDataDir = os.path.join(script_path, '..','..', 'data')
    logger.debug("set MainDataDir:\n"+MainDataDir)

# data output
output_directory = check_path_integrity(output_directory, rootDir=MainDataDir, name="output", force=True)

if DEBUG:
    logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [ ]:
# cnv data input
cnv_data_fpath = check_path_integrity(cnv_data_fpath, rootDir=MainDataDir, name="cnv data")

# genepanel data input
var_data_fpath = check_path_integrity(var_data_fpath, rootDir=MainDataDir, name="var data")

# sample info input
sample_info_fpath = check_path_integrity(sample_info_fpath, rootDir=MainDataDir, name="sample_info")

# gene info input
genes_info_fpath = check_path_integrity(genes_info_fpath, rootDir=MainDataDir, name="gene_info")


In [ ]:
# fpaths_dict
fpaths_dict = {}
fpaths_dict[genepanel_key_name] = check_path_integrity(genepanel_fpath, rootDir=MainDataDir, name="genepanel features")

for _f, _k in zip(feature_dirs, feature_key_names):
    fpath = "output/headneck/feature_selection/"+_f+"/featsel_results.csv"
    fpaths_dict[_k] = check_path_integrity(fpath, rootDir=MainDataDir, name=_k+" features")
    

In [ ]:
# load cnv data
cnv_data = pd.read_csv(cnv_data_fpath, sep='\t', header=0, index_col=0)
logger.info('loaded cnv data file with shape: '+str(cnv_data.shape))

cnv_data.columns += "__CNV"

cnv_data.head()

In [ ]:
# load genepanel data
var_data = pd.read_csv(var_data_fpath, sep='\t', header=0, index_col=0)
logger.info('loaded var data file with shape: '+str(var_data.shape))

var_data.columns += "__VAR"

var_data.head()

In [ ]:
# load info table of samples
sample_info = load_clinical(
    sample_info_fpath, **sample_info_read_csv_kwargs)
logger.info('loaded sample_info file with shape: '+str(sample_info.shape))

sample_info = sample_info.loc[cnv_data.index,:]
logger.info('keeping part of sample_infowith shape: '+str(sample_info.shape))

sample_info.head()

In [ ]:
# load info table of genes
genes_info = pd.read_csv(genes_info_fpath, sep='\t', header=0, index_col=0)
logger.info('loaded gene_info file with shape: '+str(genes_info.shape))

genes_info[gene_id_col] += "__CNV"

genes_info.head()

In [ ]:
# set the ground truth
ground_truth = sample_info.loc[cnv_data.index, sample_class_column]

ground_truth.head()

In [ ]:
# for plots
class_labels = np.array(class_labels)
class_values = np.array(class_values)

adict = define_plot_args(**cnv_plot_kwargs)
# update cnv_plot_kwargs with adict
cnv_plot_kwargs.update(adict)

adict = define_plot_args(**var_plot_kwargs)
# update cnv_plot_kwargs with adict
var_plot_kwargs.update(adict)

adict = define_plot_args(**mixed_plot_kwargs)
# update cnv_plot_kwargs with adict
mixed_plot_kwargs.update(adict)

In [ ]:
#  Plot Heatmap of genepanel_data
plot_data_heatmap(
    var_data, ground_truth, **var_plot_kwargs
)
plt.title('var data: '+str(var_data.shape[1])+' gene profiles')
save_image(saveReport=saveReport, output_directory=output_directory, img_name="heatmap_var_data", img_ext=img_ext)

In [ ]:
#  Plot Heatmap of cnv_data w/ duplicates
xlabels, xpos = get_chr_ticks(
    genes_info, cnv_data, id_col=gene_id_col, chr_col=chr_col)

plot_data_heatmap(
    cnv_data, ground_truth, xlabels, xpos, **cnv_plot_kwargs
)
plt.title('cnv data: '+str(cnv_data.shape[1])+' gene profiles')
save_image(saveReport=saveReport, output_directory=output_directory, img_name="heatmap_cnv_data", img_ext=img_ext)

In [ ]:
# remove all zero columns!
orphancols = np.where(abs(cnv_data).sum(axis=0) == 0)[0]
if len(orphancols) > 0:
    logger.warning('removing '+str(len(orphancols))+' genes from cnv data with zero columns!')
    cols2drop = cnv_data.columns.values[orphancols]
    cnv_data = cnv_data.drop(cols2drop, axis=1).copy()

# REMOVE DUPLICATES!!!!
cnv_data_uniq, dupldict, wo_dupl_set, all_dupl_set = remove_andSave_duplicates(
    cnv_data, to_compute_euclidean_distances=True,
    to_save_euclidean_distances=True, to_save_output=True,
    output_filename='cnv_data_wo_duplicates',
    output_directory=output_directory
)
single_dupl_set = set(dupldict.keys())

_countA = len(set.union(single_dupl_set, wo_dupl_set))
_countB = cnv_data_uniq.shape[1]
if not _countA == _countB:
    print(
        'ERROR: inconsistencies in the final uniq gene count!\n'+
        str(_countA)+' genes that should be in the uniq dataset VS. '+
        str(_countB)+' genes that are'
    )

In [ ]:
#  Plot Heatmap of cnv data w/o duplicates
xlabels_uniq, xpos_uniq = get_chr_ticks(
    genes_info, cnv_data_uniq, id_col=gene_id_col, chr_col=chr_col)

plot_data_heatmap(
    cnv_data_uniq, ground_truth, xlabels_uniq, xpos_uniq, **cnv_plot_kwargs
)
plt.title('cnv data w/o duplicates: '+str(cnv_data_uniq.shape[1])+' gene profiles')
save_image(saveReport=saveReport, output_directory=output_directory, img_name="heatmap_cnv_data_uniq", img_ext=img_ext)

In [ ]:
# combine var and cnv features
data_uniq = pd.concat([cnv_data_uniq, var_data], axis=1, join='inner')

In [ ]:
#  Plot Heatmap of mixed data w/o duplicates
plot_data_heatmap(
    data_uniq, ground_truth, None, None, **mixed_plot_kwargs
)
plt.title('mixed data w/o duplicates: '+str(data_uniq.shape[1])+' gene profiles')
save_image(saveReport=saveReport, output_directory=output_directory, img_name="heatmap_mixed_data_uniq", img_ext=img_ext)

In [ ]:
# split data in train-test ONCE!
stratify_by = pd.concat([ground_truth, sample_info['dataset']], axis=1, sort=False)
stratify_by = stratify_by.loc[ground_truth.index]

data_train, data_test, y_train, y_test = train_test_split(
    data_uniq, ground_truth,
    train_size=split_train_size,
    test_size=None,
    random_state=split_random_state,
    stratify=stratify_by
)

cnv_data_train = cnv_data.loc[data_train.index,:].copy()
cnv_data_test = cnv_data.loc[data_test.index,:].copy()


var_data_train = var_data.loc[data_train.index,:].copy()
var_data_test = var_data.loc[data_test.index,:].copy()

stratify_by.hist()
plt.suptitle('all '+str(ground_truth.shape[0])+' samples', fontsize=16)
save_image(saveReport=saveReport, output_directory=output_directory, img_name="stratify_by_all", img_ext=img_ext)

stratify_by.loc[cnv_data_train.index].hist()
plt.suptitle(str(y_train.shape[0])+' train samples', fontsize=16)
save_image(saveReport=saveReport, output_directory=output_directory, img_name="stratify_by_train", img_ext=img_ext)

stratify_by.loc[cnv_data_test.index].hist()
plt.suptitle(str(y_test.shape[0])+' test samples', fontsize=16)
save_image(saveReport=saveReport, output_directory=output_directory, img_name="stratify_by_test", img_ext=img_ext)


In [ ]:
xlabels_train, xpos_train = get_chr_ticks(
    genes_info, cnv_data_train, id_col=gene_id_col, chr_col=chr_col)

xlabels_test, xpos_test = get_chr_ticks(
    genes_info, cnv_data_test, id_col=gene_id_col, chr_col=chr_col)

#  Plot Heatmap of train cnv data (w/o duplicates)
plot_data_heatmap(
    cnv_data_train, y_train, xlabels_train, xpos_train, **cnv_plot_kwargs
)
plt.title('cnv train data: '+str(cnv_data_train.shape[1])+' gene profiles')
save_image(saveReport=saveReport, output_directory=output_directory, img_name="heatmap_cnv_data_train", img_ext=img_ext)

#  Plot Heatmap of test cnv data (w/o duplicates)
plot_data_heatmap(
    cnv_data_test, y_test, xlabels_test, xpos_test, **cnv_plot_kwargs
)
plt.title('cnv test data: '+str(cnv_data_test.shape[1])+' gene profiles')
save_image(saveReport=saveReport, output_directory=output_directory, img_name="heatmap_cnv_data_test", img_ext=img_ext)

In [ ]:
#  Plot Heatmap of train var data
plot_data_heatmap(
    var_data_train, y_train, None, None, **var_plot_kwargs
)
plt.title('var train data: '+str(var_data_train.shape[1])+' genes')
save_image(saveReport=saveReport, output_directory=output_directory, img_name="heatmap_var_data_train", img_ext=img_ext)

#  Plot Heatmap of test var data
plot_data_heatmap(
    var_data_test, y_test, None, None, **var_plot_kwargs
)
plt.title('var test data: '+str(var_data_test.shape[1])+' genes')
save_image(saveReport=saveReport, output_directory=output_directory, img_name="heatmap_var_data_test", img_ext=img_ext)

In [ ]:
all_cnv_data_genes = set(cnv_data.columns.values)
all_cnv_data_genes_uniq = set(cnv_data_uniq.columns.values)

all_var_data_genes = set(var_data.columns.values)

In [ ]:
features_dict = {}
features_sets = {}
for key in fpaths_dict:
    
    df = pd.read_csv(fpaths_dict[key], sep='\t', header=0, index_col=0)
    
    if genepanel_key_name in key:
        df.columns += "__VAR"
        features_sets[key] = set(df.columns.values)
        n_total = len(features_sets[key])
        n_unique = n_total
    else:
        features_dict[key] = df
        _gene_set = extract_gene_set(df)
        _gene_set = set([s + "__CNV" for s in _gene_set])      
        features_sets[key] = _gene_set
        n_total = len(features_sets[key])
        n_unique = df.shape[0]
        
    logger.info(str(n_unique)+' unique out of '+str(n_total)+' total features from '+key)

Venn diagrams to explain the functionality of the cell below:<br>
U --> data uniq genes (genes w/o dupl + single copy duplicates genes) <br>
D --> the rest of the duplicates genes copies<br>
fs --> a single features set<br>
IU --> the features that exist in the U set<br>
ID --> the features that exist in the D set<br>
NI --> the features that do NOT exist in neither set<br>
IDa --> the features that exist in the ID set and are not represented in the IU set<br>
IDb --> the features that exist in the ID set and are already represented in the IU set<br>
_ID = IDa + IDb_<br>
U_IDa --> the features that exist in the U set (but not in the IU set) and represent the IDa features<br>
**fs in data_uniq = IU + U_IDa**<br>
<img src="./files/venn_legend.jpg?1" alt="drawing" style="float:left" width="300px"/>

In [ ]:
# U: all_data_genes_uniq
# D: all_dupl_set.difference(all_data_genes_uniq)
# fs: features_sets[key]
new_features_sets = {}

for key in features_sets:
    if genepanel_key_name in key:
        U_set = all_var_data_genes
        D_set = set()
    else:
        U_set = all_cnv_data_genes_uniq
        D_set = all_dupl_set.difference(all_cnv_data_genes_uniq)
            
    fs = features_sets[key]
    _fs_original_size = len(fs)
    print(key+' feature set :')
    print('--- originally ---')
    print(' original total size: '+str(_fs_original_size))
    IU_set = fs.intersection(U_set)
    _IU_size = len(IU_set)
    print(' IU_set: '+str(_IU_size))

    ID_set = (fs.difference(IU_set)).intersection(D_set)
    _ID_size = len(ID_set)
    print(' ID_set: '+str(_ID_size))

    NI_set = (fs.difference(IU_set)).difference(ID_set)
    _NI_size =len(NI_set)
    print(' NI_set: '+str(_NI_size))

    U_IDa_set = set()
    IDa_set = set()
    IDb_set = set()
    IDc_set = set()
    done = False
    temp_set = ID_set.copy()
    while temp_set and not done:
        for ud, dl in dupldict.items():
            _IDx = set(dl).intersection(temp_set)
            if _IDx:
                temp_set = temp_set.difference(_IDx)
                if ud not in IU_set:
                    U_IDa_set.add(ud)
                    IDa_set.update(_IDx)
                else:
                    IDb_set.update(_IDx) 
        done = True

    fs_in_data_uniq = set.union(IU_set, U_IDa_set)
    _fs_in_data_uniq_size = len(fs_in_data_uniq)

    ########################################
    new_features_sets[key] = fs_in_data_uniq
    ########################################
    
    print('--- finally ---')
    print(' features in data uniq : '+str(_fs_in_data_uniq_size))
    print(' IU_set: '+str(_IU_size))
    print('   U_IDa_set: '+str(len(U_IDa_set)))
    print(' ID_set: '+str(len(ID_set)))
    print('   IDa_set: '+str(len(IDa_set)))
    print('   IDb_set: '+str(len(IDb_set)))
    print(' NI_set: '+str(_NI_size))

    if ID_set != set.union(IDa_set, IDb_set):
        print(
            'ERROR: something went wrong, ID_set != IDa_set + IDb_set, for feature set: '+key
        )
        break

    print('\n')

In [ ]:
# choose conbinations of features
feature_combinations = {
    'GnPnl_c1_OncFltNxEx': ['genepanel', 'c1_OncFltNxEx'], # 26 union
    'GnPnl_c2_ExcvFltNxEx': ['genepanel', 'c2_ExcvFltNxEx'],  # 45 union
    'GnPnl_c3_CnvNxEx': ['genepanel', 'c3_CnvNxEx'],  # 106 union
#     'c3_CNVmix': ['c3_CnvNxEx', 'c3_Cnv', 'c3_CnvMixedNxEx'], # 1 common, 1337 union
    'CNVc1c2': ['c1_OncFltNxEx', 'c2_ExcvFltNxEx'], # 45 union, 0 common
    'GnPnl_CNVc1c2': ['genepanel', 'c1_OncFltNxEx', 'c2_ExcvFltNxEx'], # 58 union
    'CNVcAll': ['c1_OncFltNxEx', 'c2_ExcvFltNxEx', 'c3_CnvNxEx'], # 127 union
    'GnPnl_CNVcAll': ['genepanel', 'c1_OncFltNxEx', 'c2_ExcvFltNxEx', 'c3_CnvNxEx'] # 140 union
}

for key in feature_combinations:
    new_features_sets[key] = set.union(*[new_features_sets[x] for x in feature_combinations[key]])

In [ ]:
fs_fprs = {}
fs_tprs = {}
fs_aucs = {}
for key in new_features_sets:
    fset = list(new_features_sets[key])
    X_train = data_train.loc[:,fset].copy()
    X_test = data_test.loc[:,fset].copy()

    # train model
    model, all_coefs, y_train_predictions, y_train_scores, fprs, tprs, interps, aucs = \
        _run_classification(
            X_train, y_train, **classification_args)

    # plot_prediction_counts_per_class
    plot_prediction_counts_per_class(
        y_train, y_train_predictions, class_labels=class_labels, class_values=class_values)
    save_image(
        saveReport=saveReport, output_directory=output_directory, 
        img_name="count_predictions_per_class", img_ext=img_ext)

    # compute_and_plot_confusion_matrices
    plt1, plt2 = compute_and_plot_confusion_matrices(
        y_train, y_train_predictions, class_labels=class_labels, class_values=class_values)
    save_image(
        saveReport=saveReport, output_directory=output_directory, 
        img_name="confusion_matrix", img_ext=img_ext, plt_obj=plt1)
    save_image(
        saveReport=saveReport, output_directory=output_directory, 
        img_name="confusion_matrix_normalized", img_ext=img_ext, plt_obj=plt2)

    # plot_roc_with_std_for_one_model
    n_splits = classification_args["n_splits"]
    plot_roc_with_std_for_one_model(n_splits, fprs, tprs, interps, aucs, figsize=(10,10), model_name=key)
    save_image(
        saveReport=saveReport, output_directory=output_directory, 
        img_name="train_crossval_roc_curves_"+key, img_ext=img_ext)

    # Test the model
    y_test_score = model.score(X_test, y_test)
    y_test_predictions = model.predict(X_test)
    y_test_predictions = pd.Series(y_test_predictions, index=X_test.index)
    y_test_predictions.name = 'test_predictions'

    plot_scatter_scores(y_train_scores, y_test_score)
    save_image(
        saveReport=saveReport, output_directory=output_directory, 
        img_name="scatter_scores", img_ext=img_ext)
    
    # prepare for the ROC curves on each feature set
    clf = CalibratedClassifierCV(base_estimator=model, cv='prefit')
    clf.fit(X_test, y_test)
    y_proba = clf.predict_proba(X_test)
#     y_proba = model.decision_function(X_test)
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1])
#     fpr, tpr, thresholds = roc_curve(y_test, y_proba)
    fs_fprs[key] = fpr
    fs_tprs[key] = tpr
    roc_auc = auc(fpr, tpr)
    fs_aucs[key] = roc_auc
    

In [ ]:
choose_models = list(new_features_sets.keys())
plot_roc_for_many_models(
    choose_models, fs_fprs, fs_tprs, fs_aucs, figsize=(10,10), 
    n_fs={key:len(new_features_sets[key]) for key in choose_models})
save_image(
    saveReport=saveReport, output_directory=output_directory, 
    img_name="test_all_models_roc_curves", img_ext=img_ext)


In [ ]:
choose_models = ['c1_OncFltNxEx', 'c2_ExcvFltNxEx', 'c3_CnvNxEx', 'CNVc1c2', 'CNVcAll']
plot_roc_for_many_models(
    choose_models, fs_fprs, fs_tprs, fs_aucs, figsize=(10,10),
    n_fs={key:len(new_features_sets[key]) for key in choose_models}
)
save_image(
    saveReport=saveReport, output_directory=output_directory, 
    img_name="test_some_models_roc_curves", img_ext=img_ext)


In [ ]:
all_aucs = pd.DataFrame.from_dict(fs_aucs, orient='index', columns=['AUC'])
choose_models = all_aucs.sort_values(by='AUC', ascending=False).iloc[:5].index.values
plot_roc_for_many_models(
    choose_models, fs_fprs, fs_tprs, fs_aucs, figsize=(10,10),
    n_fs={key:len(new_features_sets[key]) for key in choose_models}
)
save_image(
    saveReport=saveReport, output_directory=output_directory, 
    img_name="test_best_models_roc_curves", img_ext=img_ext)
